# Experimento 3 - Órdenes Correlativas

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import sys

sys.path.append("../..")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Use latex
plt.rc("text", usetex=True)

## Definición

In [ ]:
from experimento import Experimento
from src.generacion import (
    GeneradorInstancias,
    DistribucionIndependiente,
    DistribucionUniforme,
    DistribucionNormal,
)


experimento = Experimento(
    nombre="exp3",
    generador=GeneradorInstancias(
        cantidad_trabajadores=DistribucionUniforme(5, 10),
        cantidad_ordenes=DistribucionUniforme(5, 20),
        parametros_ordenes=DistribucionIndependiente(
            DistribucionNormal(10000, 2000),
            DistribucionUniforme(3, 10),
        ),
        cantidad_ordenes_correlativas=DistribucionUniforme(2, 10),
    ),
    N=1000,
    seed=42,
)

## Ejecución

In [ ]:
resultados = experimento.ejecutar()